In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from tqdm import tqdm

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ["KERAS_BACKEND"] = "jax"
import keras
import numpy as np
from sbi_mcmc.tasks import *
from sbi_mcmc.utils.bf_utils import bf_log_prob_posterior
from sbi_mcmc.utils.experiment_utils import *
from sbi_mcmc.utils.utils import *

In [ ]:
stuff = get_stuff(
    job="abi",
)
task = stuff["task"]
paths = stuff["paths"]
test_dataset = stuff["test_dataset"]
test_dataset_name = stuff["test_dataset_name"]
stats_logger = stuff["stats_logger"]
config = stuff["config"]

In [ ]:
approximator = keras.saving.load_model(paths["save_model_path"])
print(paths["save_model_path"])

In [ ]:
with stats_logger.timer("sampling"):
    abi_samples = approximator.sample(
        conditions=test_dataset, num_samples=config.target_num_draws
    )["parameters"]
    assert not np.isnan(abi_samples).any(), "NaN in abi_samples. Exiting."

In [ ]:
with stats_logger.timer("log_prob"):
    log_pdfs_abi = bf_log_prob_posterior(
        approximator, abi_samples, test_dataset["observables"]
    )

In [ ]:
result_record = {
    "abi_samples_batch": abi_samples,
    "log_pdfs_abi_batch": log_pdfs_abi,
}
save_to_file(result_record, paths["abi_result"])